In [1]:
print("***Voice Mail Text Classifcation***")

***Voice Mail Text Classifcation***


In [2]:
#Import required packages
import string
import collections
import pandas as pd 
import pyodbc
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def process_text(text, stem=True):
    """Tokenize text and stem words removing punctuation """
    text = text.translate(str.maketrans('','',string.punctuation))
    #print(text)
    tokens = word_tokenize(text)
 
    if stem:
        
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
#         lemmatizer = WordNetLemmatizer()
#         tokens = [lemmatizer.lemmatize(t) for t in tokens]
 
    return tokens

In [5]:
def vect_texts(call_texts):
    """Vectorize the texts and remove the stop words"""
    vectorizer = TfidfVectorizer(tokenizer=process_text,
                                     stop_words=stopwords.words('english'),
                                     max_df=0.5,
                                     min_df=0.1,
                                     lowercase=True,
                                ngram_range=(1, 2))

    tfidf_model = vectorizer.fit_transform(call_texts)
    return tfidf_model

In [6]:
#Commenting because server is down
"""print(pyodbc.drivers())
sqlConn = pyodbc.connect('DRIVER={xxx};SERVER=xxx;DATABASE=xxx;UID=xxx;PWD=xxx')
query = "SELECT * FROM All_leads"
primary_df = pd.read_sql(query, sqlConn)
primary_df.head(5)"""

'print(pyodbc.drivers())\nsqlConn = pyodbc.connect(\'DRIVER={xxx};SERVER=xxx;DATABASE=xxx;UID=xxx;PWD=xxx\')\nquery = "SELECT * FROM All_leads"\nprimary_df = pd.read_sql(query, sqlConn)\nprimary_df.head(5)'

In [7]:
#Read "All Leads" Data
print("Reading primary data")
primary_df = pd.read_excel("All Leads class.xlsx")
pr_df_proc = primary_df[primary_df['Description'].notna()]
pr_df_proc = pr_df_proc.reset_index()

Reading primary data


In [8]:
def prepro(text):
    """Function to preprocess Primary data"""
    pro_text = text.strip()
    pro_text = pro_text.replace('\n','')
    pro_text =pro_text.replace('\r', '')
    pro_text =pro_text.replace('\t', '')
    return pro_text

In [9]:
pr_df_proc["Proc_text"] = pr_df_proc["Description"].apply(prepro)

In [10]:
features = vect_texts(pr_df_proc["Proc_text"])

In [11]:
#Initialize list of models to be used for cross validation
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    SGDClassifier(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
print("Running 10 fold cross validation on multiple models for primary data")
CV = 10        #10 fold cross validation
cv_df = pd.DataFrame(index=range(CV * len(models)))     #Intialiaze data frame to store results
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, pr_df_proc["Class"], scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

Running 10 fold cross validation on multiple models for primary data


In [12]:
print("Model accuracy after cross validation for primary data")
print(cv_df.groupby('model_name').accuracy.mean())

Model accuracy after cross validation for primary data
model_name
LinearSVC                 0.768571
LogisticRegression        0.755238
MultinomialNB             0.755238
RandomForestClassifier    0.755238
SGDClassifier             0.651905
Name: accuracy, dtype: float64


In [13]:
def get_text(text):
    """Function to preprocess secondary data"""
    pro_text = text.split("Name/Reason:")[1]
    pro_text = text.strip()
    pro_text = pro_text.replace('\n','')
    pro_text =pro_text.replace('\r', '')
    pro_text =pro_text.replace('\t', '')
    return pro_text

In [14]:
#Read secondary data
print("Reading secondary data")
sec_df = pd.read_csv("activities-min.csv")
sec_df_call = sec_df[sec_df["Deal"].isna()]
sec_df_call = sec_df_call[sec_df_call["Class"].notna()]
sec_df_filter = sec_df_call[sec_df_call["Subject"].str.startswith("Incoming Message for:")]
sec_df_filter["Proc_text"] = sec_df_filter.Subject.apply(get_text)
sec_df_final = sec_df_filter[["Proc_text", "Class"]]
sec_df_final.reset_index().drop("index", axis=1)

Reading secondary data


,Proc_text,Class
0,"Incoming Message for: AI Health, Number: +6126...",Contact
1,"Incoming Message for: AI Australia, Number: +6...",Contact
2,"Incoming Message for: AI Health, Number: +6126...",Contact
3,"Incoming Message for: AI Health, Number: +6143...",Contact
4,"Incoming Message for: AI Australia, Number: +1...",Contact
5,"Incoming Message for: AI Australia, Number: +9...",Contact
6,"Incoming Message for: AI Australia, Number: +6...",Job
7,"Incoming Message for: AI Australia, Number: +6...",Contact
8,"Incoming Message for: AI Australia, Number: +6...",Contact
9,"Incoming Message for: AI Australia, Number: +9...",Contact


In [15]:
df_final = pr_df_proc[["Proc_text", "Class"]]

In [16]:
print("Appending Primary and Secondary data")
final_df = df_final.append(sec_df_final, ignore_index = True) 

Appending Primary and Secondary data


In [17]:
full_features = vect_texts(final_df["Proc_text"])

In [18]:
print("Running 10 fold cross validation on multiple models for full data")
CV = 10
cv_df_full = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, full_features, final_df["Class"], scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df_full = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

Running 10 fold cross validation on multiple models for full data


In [19]:
print("Model accuracy after appending secondary data")
print(cv_df_full.groupby('model_name').accuracy.mean())

Model accuracy after appending secondary data
model_name
LinearSVC                 0.640000
LogisticRegression        0.616310
MultinomialNB             0.655476
RandomForestClassifier    0.623056
SGDClassifier             0.560357
Name: accuracy, dtype: float64


In [20]:
#Split the data randomly for training and testing
print("Training models individually")
print("Spilting Train and Test data")
X_train, X_test, y_train, y_test = train_test_split(full_features, final_df["Class"], test_size=0.33, random_state=101)

Training models individually
Spilting Train and Test data


In [21]:
print("Shape of train data")
print(X_train.shape)
print("Shape of test data")
print(X_test.shape)

Shape of train data
(64, 31)
Shape of test data
(32, 31)


In [22]:
#Multinomial NB classifier
MNB_model = MultinomialNB()
print("Training Logistic regresssion model")
MNB_model.fit(X_train, y_train)
pred = MNB_model.predict(X_test) 

Training Logistic regresssion model


In [23]:
MNB_model.predict(X_test)

array(['Job', 'Job', 'Job', 'Job', 'Contact', 'Contact', 'Job', 'Contact',
       'Job', 'Contact', 'Job', 'Job', 'Contact', 'Job', 'Contact', 'Job',
       'Contact', 'Contact', 'Contact', 'Contact', 'Job', 'Job',
       'Contact', 'Job', 'Contact', 'Job', 'Contact', 'Job', 'Job', 'Job',
       'Job', 'Job'], dtype='<U9')

In [24]:
test_df = pd.DataFrame(y_test)
actual_df = test_df.join(final_df["Proc_text"])
actual_df.reset_index(inplace=True)
actual_df.drop('index', axis=1, inplace=True)
actual_df = actual_df[['Proc_text', 'Class']]

In [25]:
result_df = actual_df.join(pd.DataFrame(pred))
result_df["Match"] = result_df['Class']==result_df[0]
print(result_df.head(5))

                                           Proc_text      Class        0  \
0  Incoming Message for: AI Australia, Number: +6...    Contact      Job   
1  Dear AI Melbourne,I am a Year 10 student at No...        Job      Job   
2  Incoming Message for: AI Australia, Number: +9...    Contact      Job   
3  Hi,We can promote your website on google by do...  Marketing      Job   
4  Incoming Message for: AI Australia, Number: +6...        Job  Contact   

   Match  
0  False  
1   True  
2  False  
3  False  
4  False  


In [26]:
print("String prediction results to test.csv")
result_df.to_csv("test.csv")

String prediction results to test.csv


In [27]:
#print(confusion_matrix(y_test, pred))

In [28]:
print("Classifiaction report for Multinomial NB")
print(classification_report(y_test, pred))

Classifiaction report for Multinomial NB
             precision    recall  f1-score   support

    Contact       0.77      0.83      0.80        12
        Job       0.58      0.85      0.69        13
  Marketing       0.00      0.00      0.00         4
       Test       0.00      0.00      0.00         3

avg / total       0.52      0.66      0.58        32



In [29]:
#Multiclass logistic regression
lr_model = LogisticRegression()
print("Training Logistic regresssion model")
lr_model.fit(X_train, y_train)
pred = lr_model.predict(X_test)

Training Logistic regresssion model


In [30]:
#print(confusion_matrix(y_test, pred))
print("Classification report for Logistic regression")
print(classification_report(y_test, pred))

Classification report for Logistic regression
             precision    recall  f1-score   support

    Contact       0.71      0.83      0.77        12
        Job       0.61      0.85      0.71        13
  Marketing       0.00      0.00      0.00         4
       Test       0.00      0.00      0.00         3

avg / total       0.52      0.66      0.58        32



In [31]:
#Randomforest classifier
rf_model = RandomForestClassifier(n_estimators=200,max_depth=3, random_state=0)
print("Training Randomforest classifier")
rf_model.fit(X_train, y_train)
pred = rf_model.predict(X_test)

Training Randomforest classifier


In [32]:
#print(confusion_matrix(y_test, pred))
print("Classification report for Logistic regression")
print(classification_report(y_test, pred))

Classification report for Logistic regression
             precision    recall  f1-score   support

    Contact       0.90      0.75      0.82        12
        Job       0.55      0.92      0.69        13
  Marketing       0.00      0.00      0.00         4
       Test       0.00      0.00      0.00         3

avg / total       0.56      0.66      0.59        32



In [33]:
#Linear SVC
svc_model = LinearSVC()
print("Training Linear Support Vector Classifier")
svc_model.fit(X_train, y_train)
pred = svc_model.predict(X_test)

Training Linear Support Vector Classifier


In [34]:
#print(confusion_matrix(y_test, pred))
print("Classification report for Linear SVC")
print(classification_report(y_test, pred))

Classification report for Linear SVC
             precision    recall  f1-score   support

    Contact       0.67      0.83      0.74        12
     Invest       0.00      0.00      0.00         0
        Job       0.75      0.69      0.72        13
  Marketing       0.75      0.75      0.75         4
       Test       0.00      0.00      0.00         3

avg / total       0.65      0.69      0.66        32



In [ ]:
print("***Thank you***")
input()

***Thank you***
